This project is done on the MNIST TF dataset which consists of a GAN model.

### **1. Import Dependencies and Data**


In [ ]:
#Setting up environment.
#Installing Dependencies
# Om Sai Ram

!pip install matplotlib tensorflow-datasets ipywidgets

#(This command is needed when project is implmented locally or on jupyter notebook.)



In [ ]:
!pip list

In [ ]:
#To limit memory growth in case you are running it locally
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import tensorflow as tf
tf.config.experimental.list_physical_devices("GPU")

In [ ]:
#Bringing in rest of the dependencies
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

In [ ]:
#Downloading and importing the dataset and loading it .



ds = tfds.load("fashion_mnist",split = "train")



In [ ]:
ds.as_numpy_iterator().next()['image']

### **Visualize images and build data pipeline**

In [ ]:
#Visualizing data.
#Do some data transformation.
import numpy as np
#building an as_numpy_iterator
#OmSaiRam

In [ ]:
#Setup Connection aka iterator
dataiterator = ds.as_numpy_iterator()



In [ ]:
# Getting data out of the pipeline.
dataiterator.next()

In [ ]:
#Creating subplots

fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
#Loop Four times and get images.
for idx in range(4):
  sample = dataiterator.next()
  #squeeze just reduces it from 3 dimensional (28,28,1) to (28,28)
  ax[idx].imshow(np.squeeze(sample['image']))
  ax[idx].title.set_text(sample['label'])

In [ ]:
# scale values between 0 and 1 that are currently between 0 and 255
def scale_images(data):
  image = data['image']
  return image/255

We have to map, cache, shuffle, batch, prefetch.

Steps for building a data pipeline.

In [ ]:
#Reloaded the dataset.
ds = tfds.load('fashion_mnist',split = 'train')
#Running the dataset through the scale_images preprocessing step
ds = ds.map(scale_images)
#Cache dataset for batch
ds = ds.cache()
#Shuffle it up
ds = ds.shuffle(60000)
#Batch into 128 images per sample
ds = ds.batch(128)
#Reduce the likelihood of bottlenecking.
ds = ds.prefetch(64)

In [ ]:
ds.as_numpy_iterator().next().shape

###**Building the Neural Network.**

####**Importing Modelling Components.**

In [ ]:
#Importing the dependencies
from tensorflow.keras.models import Sequential
#Bringing in the layers for the neural network.
from tensorflow.keras.layers import Conv2D, Dense, Reshape, Flatten, LeakyReLU, Dropout, UpSampling2D


####**Building a Generator**

In [ ]:
def build_generator():
  model = Sequential()

  #Takes in random values to 7*7*128 - beginnings
  # of a generated image.
  model.add(Dense(7*7*128, input_dim = 128))
  model.add(LeakyReLU(0.2))
  model.add(Reshape((7,7,128)))

  #Upsampling block 1
  model.add(UpSampling2D())
  model.add(Conv2D(128,5,padding = 'same'))
  model.add(LeakyReLU(0.2))

  #Upsampling block 2
  model.add(UpSampling2D())
  model.add(Conv2D(128,5,padding = 'same'))
  model.add(LeakyReLU(0.2))

  # convolutional block
  model.add(Conv2D(128,4, padding = 'same'))
  model.add(LeakyReLU(0.2))

  #Convolutional block
  model.add(Conv2D(128,4, padding = 'same'))
  model.add(LeakyReLU(0.2))

  # Conv layer to get to one channel
  model.add(Conv2D(1,4,padding = 'same',activation = 'sigmoid'))
  return model

In [ ]:
generator = build_generator()

In [ ]:
generator.summary()

In [ ]:
#Testing generator
img = generator.predict(np.random.randn(4,128,1))

In [ ]:
img.shape

In [ ]:
# #Creating subplots
# #Setting up the subplot format.
# # fig is whole and ax is subplt
# fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
# #Loop Four times and get images.
# for idx, img in enumerate(img):
#   #squeeze just reduces it from 3 dimensional (28,28,1) to (28,28)
#   ax[idx].imshow(np.squeeze(img))
#   ax[idx].title.set_text(idx)

In [ ]:
img.shape

Generator part is done here.

---
---



####**Building Discriminator**

In [ ]:
def build_discriminator():
  model = Sequential()

  #First Conv Block
  model.add(Conv2D(32,5,input_shape =(28,28,1)))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  #Second Conv Block
  model.add(Conv2D(64,5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  #third Conv Block
  model.add(Conv2D(128,5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  #Fourth Conv Block
  model.add(Conv2D(256,5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  #Flatten then pass to dense layer

  model.add(Flatten())
  model.add(Dropout(0.4))
  # 1 represents false image and 0 true image
  model.add(Dense(1,activation = 'sigmoid'))

  return model

In [ ]:
discriminator = build_discriminator()

In [ ]:
discriminator.summary()

In [ ]:
img.shape

In [ ]:
discriminator.predict(img)


 img = img[0]
 As neural network expects batch if we need to pass
 image we need to do following
 discriminator.predict(np.expand_dims(img,0))

####**Custom Training Loops**

#####Setup Losses and Optimizers.

In [ ]:
# Binary Cross Entropy
# Adam is going to be optimizer for both
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
# Binary Cross Entropy is going to be loss for both.
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
#instances
#learning rate
g_opt = Adam(learning_rate = 0.0001)
d_opt = Adam(learning_rate = 0.00001)


#losses
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

#####Build Subclass Model


In [ ]:
#import model class from keras
#Importing the base model class to subclass our trianing step
from tensorflow.keras.models import Model

In [ ]:
class FashionGAN(Model):
  def __init__(self, generator, discriminator, *args, **kwargs):
    # Pass through args and kwargs to base class
    super().__init__(*args,**kwargs)

    #Create attributes for gen and discriminator
    self.generator = generator
    self.discriminator = discriminator


  def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs):
    #Compile with base class
    super().compile(*args,**kwargs)
    #Create attributes for loss and optimizers
    self.g_opt = g_opt
    self.d_opt = d_opt
    self.g_loss = g_loss
    self.d_loss = d_loss

  def train_step(self, batch):
    #get the data
    real_images = batch
    fake_images = self.generator(tf.random.normal((128,128,1)), training = False)

    #Train the discriminator
    with tf.GradientTape() as d_tape:
      #Pass the real and fake images to the discriminator model.
      yhat_real = self.discriminator(real_images,training = True)
      yhat_fake = self.discriminator(fake_images,training = True)

      yhat_realfake = tf.concat([yhat_real, yhat_fake], axis = 0)


      #Create labels for real and fake images.
      y_realfake = tf.concat([tf.zeros_like(yhat_real),tf.ones_like(yhat_fake)], axis = 0)

      #Add some noise to the outputs
      noise_real = 0.15* tf.random.uniform(tf.shape(yhat_real))
      noise_fake = -0.15* tf.random.uniform(tf.shape(yhat_fake))
      y_realfake += tf.concat([noise_real, noise_fake],axis = 0)

      #Calculate loss - BINARYCROSS
      total_d_loss = self.d_loss(y_realfake, yhat_realfake)


    #Apply Backpropogation -- nn learn
    dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables)

    self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))

    #Train the generator.
    with tf.GradientTape() as g_tape:
      #Generate some new images
      gen_images = self.generator(tf.random.normal((128,128,1)), training = True)

      #Create the predicted labels
      predicted_labels = self.discriminator(gen_images, training = False)

      # Calculate loss - trick to training to fake out the discriminator
      total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels)


    # Apply backprop
    ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)

    self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))


    return{"d_loss":total_d_loss, "g_loss":total_g_loss}


In [ ]:
#Creating instance of the subclass model
fashgan = FashionGAN(generator, discriminator)

In [ ]:
# Compile the model.
fashgan.compile(g_opt, d_opt, g_loss, d_loss)

#####**Build Callback**

In [ ]:
import os
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [ ]:
class ModelMonitor(Callback):
  def __init__(self, num_img = 3, latent_dim = 128):
    self.num_img = num_img
    self.latent_dim = latent_dim

  def on_epoch_end(self,epoch, logs = None):
    random_latent_vectors = tf.random.uniform((self.num_img, self.latent_dim, 1))
    generated_images = self.model.generator(random_latent_vectors)
    generated_images*=255
    generated_images.numpy()
    for i in range(self.num_img):
      img = array_to_img(generated_images[i])
      img.save(os.path.join('images',f'generated_img_{epoch}_{i}.png'))

#####**Train**

In [ ]:
ds.as_numpy_iterator().next()

In [ ]:
#Recommend 2000 epochs
hist = fashgan.fit(ds, epochs = 2, callbacks = [ModelMonitor()])

#####**Review Performance**

In [ ]:
plt.suptitle("Loss")
plt.plot(hist.history['d_loss'], label = 'd_loss')
plt.plot(hist.history['g_loss'], label = 'g_loss')
plt.legend()
plt.show()

####**Test Out the Generator**

#####**Generate Images**

In [ ]:

#loading the weights
# In case you want to take the pretrained model
generator.load_weights(os.path.join('', 'generatormodel.h5'))

In [ ]:
imgs = generator.predict(tf.random.normal((16,128,1)))
imgs

In [ ]:
fig, ax = plt.subplots(ncols = 8, nrows = 8, figsize = (20,20))
for r in range(8):
  for c in range(8):
    ax[r][c].imshow(imgs[(r+1)*(c+1)-1])

#####**Save the Model**

In [ ]:
generator.save('generator.h5')
discriminator.save('discriminator.h5')
